In [1]:
import numpy as np
import pandas as pd
import warnings
import pickle
import json
import requests
import time as tm
import make_dataframe as md

warnings.filterwarnings('ignore')

In [2]:
#pickle.dump(count_of_received, open('count_of_received.p', 'wb'))
count_of_received = pickle.load(open('count_of_received.p', 'rb'))

In [ ]:
count_of_received = requests.get('https://api.fda.gov/drug/event.json?search=receivedate:[20040101+TO+20190926]&count=receivedate')
count_of_received = count_of_received.json()['results']

In [3]:
def define_time_intervals(json_results):
    count = 0
    date_cutoffs = []
    for i in json_results:
        if count == 0:
            first_time = i['time']
        if (count + i['count']) < 100:
            count += i['count']
        else:
            date_cutoffs.append((first_time, i['time']))
            count = 0
    return date_cutoffs

In [5]:
time_intervals = define_time_intervals(count_of_received)

4564

In [ ]:
def get_json_data(time_intervals):
    all_data = []
    for time in time_intervals:
        url = f'https://api.fda.gov/drug/event.json?search=receivedate:[{time[0]}+TO+{time[1]}]&limit=99'
        data = requests.get(url).json()
        all_data.append(md.get_all_data(data))
    df = pd.concat(all_data)
    df['receivedate'] = pd.to_datetime(df['receivedate'])
    df = df.set_index('receivedate')
    return df

In [ ]:
# final_dataframe = get_json_data(time_intervals)
# pickle.dump(final_dataframe, open('final_dataframe.p', 'wb'))

In [ ]:
final_dataframe.shape